<a href="https://colab.research.google.com/github/isaacfinberg/Novozymes-Enzyme-Stability-Prediction/blob/develop/AlphaFold_10_27_Louis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/isaacfinberg/Novozymes-Enzyme-Stability-Prediction/main/MutationsDeltaTM.csv')
df

,Unnamed: 0,mutated_sequence,pdb_id,wt_sequence,mutations,delta_tm
0,0,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,2LZM,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,W138Y,-6.3
1,1,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,2LZM,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,"W138Y,W126Y,W158Y",-6.2
2,2,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,2LZM,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,"W138Y,W126Y,W158Y",-6.6
3,3,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,2LZM,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,E128A,0.6
4,4,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,2LZM,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,V131A,1.0
...,...,...,...,...,...,...
4377,993,MQTLLVSSLVVSLAAALPHYIRSNGIEASLLTDPKDVSGRTVDYII...,1CF3,MQTLLVSSLVVSLAAALPHYIRSNGIEASLLTDPKDVSGRTVDYII...,V442G,2.0
4378,994,MQTLLVSSLVVSLAAALPHYIRSNGIEASLLTDPKDVSGRTVDYII...,1CF3,MQTLLVSSLVVSLAAALPHYIRSNGIEASLLTDPKDVSGRTVDYII...,S444N,3.0
4379,995,MQTLLVSSLVVSLAAALPHYIRSNGIEASLLTDPKDVSGRTVDYII...,1CF3,MQTLLVSSLVVSLAAALPHYIRSNGIEASLLTDPKDVSGRTVDYII...,D473N,4.0
4380,996,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,2LZM,MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,W138Y,-6.3


In [ ]:
!pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
!pip install -q "jax[cuda11_cudnn805]>=0.3.8,<0.4" -f https://storage.googleapis.com/jax-releases/jax_releases.html

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
PATH_PREFIX = '/content/gdrive/MyDrive/Kaggle/AlphaFold'
jobname = 'test'
queries_path=f"{PATH_PREFIX}/{jobname}.csv"

use_amber = False
custom_template_path = None
use_templates = False
msa_mode = "MMseqs2 (UniRef+Environmental)" 
a3m_file = f"{jobname}.a3m"
pair_mode = "unpaired+paired"
model_type = "AlphaFold2-ptm" 
num_recycles = 3
save_to_google_drive = False
dpi = 200

In [ ]:
from google.colab import files
import os.path
import re
import hashlib
import random
import sys
import warnings

wt_done = {}
job_paths = []

for [i, row] in df.iterrows():

  pdb_id = row['pdb_id']
  mutations = row['mutations']
  job_name = (pdb_id + '_WT').upper()

  if job_name in wt_done:
    continue

  wt_done[job_name] = True

  query_sequence = row['wt_sequence']
  query_sequence = "".join(query_sequence.split())
  
  job_path = PATH_PREFIX + '/' + job_name
  job_paths.append(job_path)

  with open(f"{job_path}.csv", "w") as text_file:
    text_file.write(f"id,sequence\n{job_name},{query_sequence}")

In [ ]:
from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt

def prediction_callback(unrelaxed_protein, length, prediction_result, input_features, type):
  print('Done prediction')
  #fig = plot_protein(unrelaxed_protein, Ls=length, dpi=150)
  #plt.show()
  #plt.close()

model_type = set_model_type(is_complex, model_type)
download_alphafold_params(model_type, Path("."))

In [ ]:
result_dir = PATH_PREFIX + '/Results'

for job_path in job_paths:
    
  queries, is_complex = get_queries(f"{job_path}.csv")

  run(
      queries=queries,
      result_dir=result_dir,
      use_templates=use_templates,
      custom_template_path=custom_template_path,
      use_amber=use_amber,
      msa_mode=msa_mode,    
      model_type=model_type,
      num_models=5,
      num_recycles=num_recycles,
      model_order=[1,2,3,4,5],
      is_complex=is_complex,
      data_dir=Path("."),
      keep_existing_results=False,
      recompile_padding=1.0,
      rank_by="auto",
      pair_mode=pair_mode,
      stop_at_score=float(100),
      prediction_callback=prediction_callback,
      dpi=dpi
  )

2022-10-27 03:21:16,384 Found 5 citations for tools or databases
2022-10-27 03:21:21,759 Query 1/1: 2LZM_WT (length 164)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:00 remaining: 00:00]


2022-10-27 03:21:22,876 Running model_1
2022-10-27 03:22:36,044 model_1 took 70.9s (3 recycles) with pLDDT 95.9 and ptmscore 0.879
Done prediction
2022-10-27 03:22:39,736 Running model_2
2022-10-27 03:23:35,053 model_2 took 52.9s (3 recycles) with pLDDT 96.6 and ptmscore 0.898
Done prediction
2022-10-27 03:23:38,740 Running model_3
2022-10-27 03:24:33,608 model_3 took 52.6s (3 recycles) with pLDDT 96 and ptmscore 0.881
Done prediction
2022-10-27 03:24:37,247 Running model_4
2022-10-27 03:25:32,038 model_4 took 52.5s (3 recycles) with pLDDT 96.8 and ptmscore 0.899
Done prediction
2022-10-27 03:25:35,807 Running model_5
2022-10-27 03:26:30,654 model_5 took 52.6s (3 recycles) with pLDDT 96.9 and ptmscore 0.9
Done prediction
2022-10-27 03:26:34,352 reranking models by plddt
2022-10-27 03:26:36,189 Done
2022-10-27 03:26:36,206 Found 5 citations for tools or databases
2022-10-27 03:26:40,842 Query 1/1: 2RN2_WT (length 155)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:00 remaining: 00:00]


2022-10-27 03:26:43,157 Running model_1
2022-10-27 03:28:07,786 model_1 took 79.0s (3 recycles) with pLDDT 94.1 and ptmscore 0.891
Done prediction
2022-10-27 03:28:15,821 Running model_2
2022-10-27 03:29:06,887 model_2 took 45.8s (3 recycles) with pLDDT 94.3 and ptmscore 0.9
Done prediction
2022-10-27 03:29:10,403 Running model_3
2022-10-27 03:30:01,514 model_3 took 45.7s (3 recycles) with pLDDT 93.9 and ptmscore 0.892
Done prediction
2022-10-27 03:30:05,090 Running model_4
2022-10-27 03:30:56,212 model_4 took 45.8s (3 recycles) with pLDDT 91.3 and ptmscore 0.862
Done prediction
2022-10-27 03:30:59,748 Running model_5
2022-10-27 03:31:50,779 model_5 took 45.7s (3 recycles) with pLDDT 92.2 and ptmscore 0.875
Done prediction
2022-10-27 03:31:54,252 reranking models by plddt
2022-10-27 03:31:57,382 Done
2022-10-27 03:31:57,400 Found 5 citations for tools or databases
2022-10-27 03:32:02,033 Query 1/1: 1PML_WT (length 562)


...

2022-10-27 18:22:58,802 Sleeping for 10s. Reason: RUNNING


RUNNING: |          | 169/? [elapsed: 02:55 remaining: 00:00]

2022-10-27 18:23:09,075 Sleeping for 6s. Reason: RUNNING


COMPLETE: |          | 169/? [elapsed: 03:03 remaining: 00:00]


2022-10-27 18:23:29,202 Running model_1


In [ ]:
import py3Dmol
import glob
import matplotlib.pyplot as plt
from colabfold.colabfold import plot_plddt_legend
rank_num = 1
color = "lDDT" 
show_sidechains = False 
show_mainchains = False 

pdb_filename = f"{jobname}_unrelaxed_rank_{rank_num}_model_*.pdb"
pdb_file = glob.glob(pdb_filename)
model_name = f"rank_1"
model_data = open(pdb_file[0],'r').read()

print(model_data)

MODEL     1                                                                     
ATOM      1  N   PRO A   1       4.113   2.954 -13.940  1.00 97.40           N  
ATOM      2  CA  PRO A   1       2.785   2.550 -13.474  1.00 97.40           C  
ATOM      3  C   PRO A   1       2.514   2.968 -12.030  1.00 97.40           C  
ATOM      4  CB  PRO A   1       2.814   1.025 -13.605  1.00 97.40           C  
ATOM      5  O   PRO A   1       3.404   2.880 -11.181  1.00 97.40           O  
ATOM      6  CG  PRO A   1       3.924   0.745 -14.566  1.00 97.40           C  
ATOM      7  CD  PRO A   1       4.994   1.781 -14.374  1.00 97.40           C  
ATOM      8  N   ILE A   2       1.404   3.595 -11.900  1.00 98.43           N  
ATOM      9  CA  ILE A   2       1.032   4.026 -10.557  1.00 98.43           C  
ATOM     10  C   ILE A   2      -0.196   3.250 -10.086  1.00 98.43           C  
ATOM     11  CB  ILE A   2       0.756   5.546 -10.508  1.00 98.43           C  
ATOM     12  O   ILE A   2  

In [9]:
!pip install py3Dmol
!pip install colabfold
import py3Dmol
import glob
import matplotlib.pyplot as plt
from colabfold.colabfold import plot_plddt_legend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
pdb_file = '/content/gdrive/MyDrive/Kaggle/AlphaFold/Results/3TIM_WT_unrelaxed_rank_2_model_3.pdb'

def show_pdb(rank_num=1, show_sidechains=False, show_mainchains=False, color="lDDT"):
  model_name = f"rank_{rank_num}"
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb_file,'r').read(),'pdb')

  if color == "lDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':50,'max':90}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    chains = len(queries[0][1]) + 1 if is_complex else 1
    for n,chain,color in zip(range(chains),list("ABCDEFGH"),
                     ["lime","cyan","magenta","yellow","salmon","white","blue","orange"]):
      view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})  
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  view.zoomTo()
  return view

show_pdb().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
if msa_mode == "custom":
  print("Don't forget to cite your custom MSA generation method.")

!zip -FSr $jobname".result.zip" config.json $jobname*".json" $jobname*".a3m" $jobname*"relaxed_rank_"*".pdb" "cite.bibtex" $jobname*".png"
files.download(f"{jobname}.result.zip")

if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  uploaded.SetContentFile(f"{jobname}.result.zip")
  uploaded.Upload()
  print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")